In [7]:
from requests_html import HTMLSession
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta
import concurrent.futures


In [8]:
#public client ID
client_id = "OrZoy8IshHh0YwFZmmXWC0ORwJNEKm9Z" 

# search playlist api url format like below:
# https://api-v2.soundcloud.com/search/playlists_without_albums?q=<keyword>&client_id=<client_id>&limit=<so tu nhien trong khoan 0..200>
# url = f'https://api-v2.soundcloud.com/search/playlists_without_albums?q={keyword}&client_id={client_id}'

# session = requests.session()
session = HTMLSession()

# I use HTMLSession instead of requests.session 
# cause it produce much less fail requests. 
# About more than 9 failures for requests.session but when I use HTMLSession there no fail requests.


collection_list = []

# loop to get playlist id, keywords used to seach are a,b,c..
def get_collections(keyword_number):
    keyword = chr(keyword_number + 97) # 97 is a in ascii

    url = f'https://api-v2.soundcloud.com/search/playlists_without_albums?q={keyword}&client_id={client_id}&limit=100'
    r = session.get(url)

    if r.status_code != 200:
        time.sleep(0.5)
        fail+=1
        return

    content = r.json()
    collection = pd.json_normalize(content["collection"])
    collection_list.append(collection)


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(get_collections, range(10))

collection_df = pd.concat(collection_list)
collection_df

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,user.visuals,user.badges.pro,user.badges.pro_unlimited,user.badges.verified,user.station_urn,user.station_permalink,user.visuals.urn,user.visuals.enabled,user.visuals.visuals,user.visuals.tracking
0,https://i1.sndcdn.com/artworks-000134890657-dq...,2014-09-22T05:20:11Z,Kickback and chill.,5864037,all,Chillout,51957607,playlist,,2021-08-21T06:56:42Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:81...,artist-stations:81561939,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/artworks-000128912780-0c...,2015-09-08T18:51:18Z,Music you can play at anytime for any occasion.,21269204,all,electronic,142851503,playlist,,2021-11-22T02:23:51Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:14...,artist-stations:14702530,soundcloud:users:14702530,True,"[{'urn': 'soundcloud:visuals:2306244', 'entry_...",NaN
2,https://i1.sndcdn.com/artworks-000401975853-xp...,2018-09-08T15:04:01Z,"autumn cosy night, sweater weather\n\n+) I'd l...",3230461,all,,598944738,playlist,None,2021-01-10T03:19:06Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:15...,artist-stations:150205120,soundcloud:users:150205120,True,"[{'urn': 'soundcloud:visuals:52703776', 'entry...",NaN
3,None,2018-02-03T13:48:36Z,None,32716825,all,,439472172,playlist,None,2019-05-26T16:13:00Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:77...,artist-stations:77546952,soundcloud:users:77546952,True,"[{'urn': 'soundcloud:visuals:98687013', 'entry...",NaN
4,None,2018-11-24T19:02:21Z,None,5407574,all,,650533902,playlist,None,2019-09-30T14:16:21Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:22...,artist-stations:224470945,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,2018-06-11T06:40:01Z,None,106878778,all,,538271169,playlist,None,2020-12-14T19:24:57Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:35...,artist-stations:357715568,NaN,NaN,NaN,NaN
96,None,2016-06-09T16:08:44Z,None,5149414,all,,232294030,playlist,None,2016-06-11T08:19:15Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:30...,artist-stations:30284210,soundcloud:users:30284210,True,"[{'urn': 'soundcloud:visuals:10630023', 'entry...",NaN
97,None,2017-06-11T21:50:00Z,None,58765307,all,,329936852,playlist,None,2017-06-11T21:50:00Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:25...,artist-stations:256153933,NaN,NaN,NaN,NaN
98,https://i1.sndcdn.com/artworks-000207607822-7y...,2014-05-11T12:03:04Z,DOWNLOAD and USE this music in your videos. (Y...,7921942,all,Instrumental,34416293,playlist,,2021-02-04T12:13:08Z,...,NaN,False,False,False,soundcloud:system-playlists:artist-stations:20...,artist-stations:20946955,soundcloud:users:20946955,True,"[{'urn': 'soundcloud:visuals:14121398', 'entry...",NaN


In [9]:
playlist_id_series = collection_df['id']
playlist_id_series = playlist_id_series.drop_duplicates()
playlist_id_series

0      51957607
1     142851503
2     598944738
3     439472172
4     650533902
        ...    
95    538271169
96    232294030
97    329936852
98     34416293
99    231852356
Name: id, Length: 1000, dtype: int64

In [10]:
i = 0
playlist_list = []
user_list = []
track_list = []

def collect_data(i):
    id = playlist_id_series.iloc[i]
    url = f'https://api-v2.soundcloud.com/playlists/{id}?client_id={client_id}'

    r = session.get(url)
    content = r.json()
    playlist = pd.json_normalize(content)
    playlist.drop(columns = list(playlist.filter(regex = r'user\..*')), inplace = True)
    playlist_list.append(playlist)

    user = pd.json_normalize(content['user'])
    user_list.append(user)

    tracks = pd.json_normalize(content['tracks'])
    if not len(tracks):
        return

    tracks = tracks[pd.notna(tracks['artwork_url'])]
    track_list.append(tracks)


with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        executor.map(collect_data, range(1000))

playlist_df = pd.concat(playlist_list)
user_df = pd.concat(user_list)
track_df = pd.concat(track_list)

In [11]:
tracks_df = track_df.drop_duplicates('id')
user_df = user_df.drop_duplicates('id')

In [12]:
!mkdir ApiData
playlist_df.to_csv('ApiData/playlist.csv', sep = '\t', index = False)
user_df.to_csv('ApiData/user.csv' , sep = '\t', index = False)
track_df.to_csv('ApiData/track.csv', sep = '\t', index = False)

A subdirectory or file ApiData already exists.
